In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# === Load pre-trained model ===
model = load_model('best_emotion_model.keras')

# === Emotion labels ===
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# === Load OpenCV face detector ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === Preprocess image (detect face, crop, resize, normalize) ===
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Unable to load image file.")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    face_detected = False
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        gray_face = gray[y:y+h, x:x+w]
        face_detected = True
    else:
        gray_face = gray  # Just use full image, no pop-up alert

    gray_face = cv2.resize(gray_face, (48, 48))
    gray_face = gray_face.astype('float32') / 255.0
    gray_face = np.expand_dims(gray_face, axis=(0, -1))

    return gray_face, img, faces, face_detected

# === Predict emotion ===
def predict_emotion(img_path):
    img_data, original_img, faces, face_detected = preprocess_image(img_path)
    preds = model.predict(img_data)
    emotion_idx = np.argmax(preds)
    confidence = preds[0][emotion_idx]
    return emotion_labels[emotion_idx], confidence, original_img, faces, face_detected

# === Open and display image + prediction ===
def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        try:
            # Predict emotion
            emotion, confidence, original_img, faces, face_detected = predict_emotion(file_path)

            # Draw face box if detected
            display_img = original_img.copy()
            if face_detected:
                (x, y, w, h) = faces[0]
                cv2.rectangle(display_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(display_img, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36,255,12), 2)

            # Convert BGR → RGB for Tkinter
            display_img = cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB)
            display_img = cv2.resize(display_img, (250, 250))
            img_pil = Image.fromarray(display_img)
            img_tk = ImageTk.PhotoImage(img_pil)

            # Update GUI
            image_label.config(image=img_tk)
            image_label.image = img_tk
            result_label.config(
                text=f"Emotion: {emotion}\nConfidence: {confidence * 100:.2f}%",
                fg="#007ACC"
            )

        except Exception as e:
            messagebox.showerror("Error", f"Prediction error:\n{str(e)}")

# === Tkinter GUI Setup ===
root = tk.Tk()
root.title("Face Emotion Detection")
root.geometry("340x470")
root.resizable(False, False)
root.configure(bg="#f2f2f2")

# === UI Elements ===
title_label = tk.Label(root, text="Emotion Detection", font=("Arial", 18, "bold"), bg="#f2f2f2", fg="#333")
title_label.pack(pady=15)

btn_open = tk.Button(root, text="Open Photo", command=open_image, font=("Arial", 12), bg="#007ACC", fg="white", width=15)
btn_open.pack(pady=10)

image_label = tk.Label(root, bg="#f2f2f2")
image_label.pack(pady=10)

result_label = tk.Label(root, text="Prediction result will appear here", font=("Arial", 13), bg="#f2f2f2", fg="#555")
result_label.pack(pady=10)

footer_label = tk.Label(root, text="Developed by Mohamed Alaa", font=("Arial", 9, "italic"), bg="#f2f2f2", fg="#777")
footer_label.pack(side="bottom", pady=5)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
